# Hubbard Hamiltonian

I'm planning on running VQE and Trotter simulation on a Hubbard Hamiltonian to better understand its structure. 

In [7]:
import numpy as np
from utils import *

## Defining creation and annihilation operators 

In [2]:
class Operator: 
    def __init__(self, annihilation, spin): 
        self.annihilation = annihilation 
        self.spin = spin 
        
class CreationOperator:
    def __init__(self, spin, qubit, dim):
        self.type = 'creation'
        self.spin = spin
        self.qubit = qubit
        self.dim = dim

class AnnihilationOperator:
    def __init__(self, spin, qubit, dim):
        self.type = 'annihilation'
        self.spin = spin
        self.qubit = qubit
        self.dim = dim

In [3]:
c_1 = CreationOperator('down', 0, 5)
a_1 = AnnihilationOperator('up', 3, 5)

## Jordan-Wigner Transformation

In [4]:
def encode_JW(operator):
    if type(operator) != CreationOperator and type(operator) != AnnihilationOperator:
        raise Error("operator must be a CreationOperator or AnnihilationOperator")
        
    result = np.array([[1.0]], dtype='complex128') 
    string_repr = ''
    # Notice the order of the tensor product: gate on 0th qubit is "leftmost" in product
    for i in range(operator.dim): 
        if i < operator.qubit: 
            result = NKron(result, Z)
            string_repr = string_repr + 'Z'
        elif i == operator.qubit and operator.type == 'creation': 
            result = NKron(result, (X -1j*Y) / 2)
            string_repr = string_repr + '-'
        elif i == operator.qubit and operator.type == 'annihilation': 
            result = NKron(result, (X +1j*Y) / 2) 
            string_repr = string_repr + '+'
        elif i > operator.qubit: 
            result = NKron(result, I) 
            string_repr = string_repr + 'I'
        else: 
            raise Error("Something is wrong with this code!")
    #print(string_repr)
    return result

In [5]:
x = encode_JW(c_1)

In [6]:
x.shape

(32, 32)

## Defining the Hubbard Hamiltonian

Okay, now that we have those preliminaries out of the way, what is the Hubbard Hamiltonian? What's the motivation behind it? How is it useful? 

Answer these. 

### 2D Hubbard Hamiltonian on a Square Lattice 

Here is a model of a 2D Hubbard Hamiltontian on a square lattice from [1811.04476](https://arxiv.org/pdf/1811.04476.pdf). We'll represent it by a 2D array: 2D to get the square structure, and in each square `[i][j]`, a dictionary of 2 elements: 'creation' and 'annihilation'. 

![](2d_hubbard.png)

In [47]:
N_X = 5 
N_Y = 5
N = N_X * N_Y
square_lattice = [[ {'i': i, 'j': j, 
                     'creation': CreationOperator('UP', N_Y * i + j, N), 
                     'annihilation': AnnihilationOperator('UP', N_Y * i + j, N) 
                    } for j in range(N_X)] for i in range(N_Y)]

In [50]:
square_lattice[0][3]['creation'].qubit

3